In [16]:
#!pipenv install numpy --dev

In [17]:
#!pipenv install pandas --dev

In [1]:
%load_ext blackcellmagic

In [2]:
#!pipenv install black=="19.3b0" blackcellmagic --dev

In [1]:
import numpy as np

In [4]:
import vaex
from ast import literal_eval

In [5]:
df = vaex.read_csv_and_convert("million.9_song.csv")
df = vaex.open("million.9_song.csv.hdf5")

In [6]:
def list_check(i):
    if type(i) == list:
        return i


def extract_links(list_of_tuples):
    """
    1. List of tuples was saved as a string, turn it back into a list
    2. Turn Tuples into list. Output is a list of strings & lists
    3. filter only the lists, drop the identifiers such as "Written by". Output is a list of lists. 
    4. Turn the list of lists into one list
        4a. Remove stuff from the url stuff from the song url to get an identifier.
    """
    a = literal_eval(list_of_tuples)
    a = list(filter(list_check, [i[1] for i in a]))
    a = [
        j.replace("https://genius.com/artists/", "")
        .replace("https://genius.com/", "")
        .replace("artists/", "")
        .replace("-lyrics", "")
        .lower()
        for i in a
        for j in i
    ]
    return a
def get_header_links(item):
    """
    1. List was saved as a string. turn it back into a list
    2. 
    """
    item = literal_eval(item)
    if (type(item) == list) &( len(item)>0):
        item = item[0].replace("https://genius.com/artists/", "")
        item = item.lower()
        return [item]

In [7]:
df.drop(df.index,inplace=True)

#,song_num,title,url,lyrics,side_table,header_links
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"'[""I\'ve got nowhere left to hide"", \'No time left...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids']
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"""['[Hook]', 'You can catch me rollin wit my goon...","[('Release Date', [])]",['https://genius.com/artists/Caution']
2,358464,“Ananseem Stories”,'https://genius.com/Freeman-dwamena-ananseem-sto...,"""['There is in Ghana story-feeling art called An...","[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena']
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"'[\'[Verse 1]\', ""You don\'t see me"", \'Now I am far...","[('Release Date', [])]",['https://genius.com/artists/Kazaky']
4,358510,Amused,'https://genius.com/Jimmy-the-rapper-and-jay-bel...,"""['[Verse 1 - Jimmy The Rapper]', 'Came came, sa...","[('Release Date', [])]","""['https://genius.com/artists/Jimmy-the-rapper-a..."
...,...,...,...,...,...,...
"1,907,745",2391271,Bem Estar,https://genius.com/3030-bem-estar-lyrics,"""['[Refrão: LK e Felp]', 'Eu quero seu bem estar...","""[('Written By', ['https://genius.com/artists/Fe...",['https://genius.com/artists/3030']
"1,907,746",2391184,Vitamine 30,'https://genius.com/Alireza-jj-and-sijal-vitamin...,"""['؛[مقدمه: بهزاد لیتو]؛', 'زندگیا شده دیگه بس ک...","""[('Written By', ['https://genius.com/artists/Si...","""['https://genius.com/artists/Alireza-jj-and-sij..."
"1,907,747",2391236,#AlleWollenHaben,https://genius.com/Haben-allewollenhaben-lyrics,"""['[Intro]', 'Was soll ich dir sagen, Bruder?', ...","[('Release Date', [])]",['https://genius.com/artists/Haben']
"1,907,748",2391263,Giovanni and I,'https://genius.com/Dominick-rivera-giovanni-and...,"""['I loved the way', 'I called, you came', 'Reme...",[],['https://genius.com/artists/Dominick-rivera']


In [8]:
df.add_virtual_column("meta_data_1",df.side_table.apply(extract_links))
df.add_virtual_column("fixed_header_links", df.header_links.apply(get_header_links))

In [9]:
md1 = df.evaluate(df.meta_data_1)
h = df.evaluate(df.fixed_header_links)

In [10]:
def return_longest(a,b):
    if (type(a)==list) & (type(b)==list):
        if len(a)>len(b):
            return a
        else:
            return b
    elif type(b)!=list:
        return a

In [11]:
df.add_column("meta_data_2",np.array([return_longest(a,b) for a,b in zip(md1,h)]))

In [12]:
df.add_virtual_column(
    "song_identifier",
    df.url.str.replace("https://genius.com/", "")
    .str.replace("-lyrics", "")
    .str.replace("https://genius.com/artists/", "")
    .str.lower()
)

In [13]:
from collections import Counter
si_count = Counter([i for i in df.song_identifier.evaluate() ])

In [14]:
#checks for duplicates
si_count.most_common(50)

[('bispo-olho-gordo', 2),
 ('the-cloud-cars-old-die-young', 2),
 ('lloyd-all-of-me', 2),
 ('yo-yo-honey-singh-blue-eyes', 2),
 ('krayzie-bone-clash-of-the-titans-rap-god-remix', 2),
 ('dealema-nada-dura-para-sempre', 2),
 ('cheesecake-dave-colours-remix', 2),
 ('parachute-what-i-know', 2),
 ('apologetix-sufferin-just-finished', 2),
 ('will-young-dance-the-night-away', 2),
 ('the-cure-just-one-kiss', 2),
 ('halifax-the-next-two-weeks', 2),
 ('teefreeze-solo', 2),
 ('atb-everything-is-wrong', 2),
 ('don-tetto-ha-vuelto-a-suceder', 2),
 ('paulina-rubio-pobre-nina-rica', 2),
 ('after-forever-monolith-of-doubt', 2),
 ('paul-oakenfold-ready-steady-go', 2),
 ('paul-gilbert-i-am-satan', 2),
 ('emerson-lake-and-palmer-a-time-and-a-place', 2),
 ('flow-projectz-toast-to-tha-future', 2),
 ('mark-owen-pieces-of-heaven', 2),
 ('red-flag-a-life-worth-living', 2),
 ('nat-king-cole-april-in-paris', 2),
 ('nat-king-cole-i-hear-music', 2),
 ('sissel-lr-meg-a-kjenne', 2),
 ('the-juan-maclean-you-were-a-ru

# Bag of Words vectorization for song meta data

Networkx graph is coming out way too small. I have over 31 million edges. But the graph has 393,971 nodes

### why list comprehension
list comprehesion worked where numpy vectorization and vaex built in string operations failed.

In [ ]:
d = df.meta_data_2.evaluate()

In [ ]:
data_strings = [" ".join(i) for i in d]

### Build vocabulary
countvectorizer() turns `'pete-rock'` into `['pete','rock']`
So we need to build a new vocabulary

In [ ]:
data_vocabulary = list(" ".join(data_strings).split(" "))

In [ ]:
len(data_vocabulary)

In [ ]:
len(set(data_vocabulary))

# 9 million features condenses into 24k nodes
maybe the edgelist wasn't that far off

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
)

In [ ]:
vectorizer.vocabulary =set(data_vocabulary)

In [ ]:
X = vectorizer.fit_transform(data_strings)

In [ ]:
len(vectorizer.vocabulary_)

### Back to Vaex
This is going to be very round-about-way.

This did not work, skip to truncated SVD for now.

In [ ]:
# import pyarrow
# pyarrow.array(X)

### Truncated SVD
used because the truncated form can take sparse matrices.

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, n_iter=30, random_state=42)

In [ ]:
%%time
x_svd = svd.fit_transform(X)  

In [ ]:
x_svd.shape

# Get lyric embeddings

In [15]:
npl = df.lyrics.evaluate()

In [21]:
import re


def clean_lyrics(lyrics):
    """
    0. Check for type == string, or else it will throw an error. didn't use a plain else incase there were other errors.
    1. turn the pickled list into into an actual list
    2. rejoin the list into one string
    3. regex and other string operations to clean the lyrics
    """
    try:
        lyrics = literal_eval(lyrics)
        l = " ".join(lyrics)
        l = re.sub(r'\[(.+?)\]|"', " ", l)
        l = (
            l.replace("'", "")
            .replace(r", ", " ")
            .replace("(", "")
            .replace(")", "")
            .replace("?", "")
            .replace(":" "", "")
            .strip()
            .lower()
        )
        return l
    except:
        return  " "

In [23]:
df.add_virtual_column("clean_lyrics", df.lyrics.apply(clean_lyrics))

I should definitely have been visualizing data earlier. There are a few outliers causing problems. no lyrics...

I was way too confident in my data.

In [24]:
df

#,song_num,title,url,lyrics,side_table,header_links,meta_data_1,fixed_header_links,meta_data_2,song_identifier,clean_lyrics
0,358479,Hostage,https://genius.com/Dangerkids-hostage-lyrics,"'[""I\'ve got nowhere left to hide"", \'No time left...","[('Release Date', [])]",['https://genius.com/artists/Dangerkids'],[],['dangerkids'],['dangerkids'],dangerkids-hostage,'ive got nowhere left to hide no time left for a...
1,358490,Fuck Em All,https://genius.com/Caution-fuck-em-all-lyrics,"""['[Hook]', 'You can catch me rollin wit my goon...","[('Release Date', [])]",['https://genius.com/artists/Caution'],[],['caution'],['caution'],caution-fuck-em-all,'you can catch me rollin wit my goonies we be wi...
2,358464,“Ananseem Stories”,'https://genius.com/Freeman-dwamena-ananseem-sto...,"""['There is in Ghana story-feeling art called An...","[('Release Date', [])]",['https://genius.com/artists/Freeman-dwamena'],[],['freeman-dwamena'],['freeman-dwamena'],freeman-dwamena-ananseem-stories-annotated,'there is in ghana story-feeling art called anan...
3,358434,The Sun,https://genius.com/Kazaky-the-sun-lyrics,"'[\'[Verse 1]\', ""You don\'t see me"", \'Now I am far...","[('Release Date', [])]",['https://genius.com/artists/Kazaky'],[],['kazaky'],['kazaky'],kazaky-the-sun,'you dont see me now i am far away you dont feel...
4,358510,Amused,'https://genius.com/Jimmy-the-rapper-and-jay-bel...,"""['[Verse 1 - Jimmy The Rapper]', 'Came came, sa...","[('Release Date', [])]","""['https://genius.com/artists/Jimmy-the-rapper-a...",[],['jimmy-the-rapper-and-jay-bel'],['jimmy-the-rapper-and-jay-bel'],jimmy-the-rapper-and-jay-bel-amused,'came came saw saw conquered your body.. frame b...
...,...,...,...,...,...,...,...,...,...,...,...
"1,907,745",2391271,Bem Estar,https://genius.com/3030-bem-estar-lyrics,"""['[Refrão: LK e Felp]', 'Eu quero seu bem estar...","""[('Written By', ['https://genius.com/artists/Fe...",['https://genius.com/artists/3030'],"['felp22', 'rod-3030', 'luan-lk']",['3030'],"['felp22', 'rod-3030', 'luan-lk']",3030-bem-estar,'eu quero seu bem estar mas na minha cama você j...
"1,907,746",2391184,Vitamine 30,'https://genius.com/Alireza-jj-and-sijal-vitamin...,"""['؛[مقدمه: بهزاد لیتو]؛', 'زندگیا شده دیگه بس ک...","""[('Written By', ['https://genius.com/artists/Si...","""['https://genius.com/artists/Alireza-jj-and-sij...","""['sijal', 'sohrab-mj', 'alireza-jj', 'momet-sha...",['alireza-jj-and-sijal'],"""['sijal', 'sohrab-mj', 'alireza-jj', 'momet-sha...",alireza-jj-and-sijal-vitamine-30,'؛ ؛ زندگیا شده دیگه بس که بیخودی همه با خیال با...
"1,907,747",2391236,#AlleWollenHaben,https://genius.com/Haben-allewollenhaben-lyrics,"""['[Intro]', 'Was soll ich dir sagen, Bruder?', ...","[('Release Date', [])]",['https://genius.com/artists/Haben'],[],['haben'],['haben'],haben-allewollenhaben,'was soll ich dir sagen bruder du weißt alter al...
"1,907,748",2391263,Giovanni and I,'https://genius.com/Dominick-rivera-giovanni-and...,"""['I loved the way', 'I called, you came', 'Reme...",[],['https://genius.com/artists/Dominick-rivera'],[],['dominick-rivera'],['dominick-rivera'],dominick-rivera-giovanni-and-i,'i loved the way i called you came remember thos...


In [ ]:
cl = df.clean_lyrics.evaluate()

In [ ]:
df.head(5)

In [25]:
import matplotlib.pyplot
df.plot1d(df.clean_lyrics.str.len())

ERROR:MainThread:vaex.execution:error in task, flush task queue
Traceback (most recent call last):
  File "/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/vaex/scopes.py", line 86, in evaluate
    result = self[expression]
  File "/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/vaex/scopes.py", line 133, in __getitem__
    raise KeyError("Unknown variables or column: %r" % (variable,))
KeyError: "Unknown variables or column: 'str_len(clean_lyrics)'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/vaex/scopes.py", line 86, in evaluate
    result = self[expression]
  File "/home/cwcollins/.local/share/virtualenvs/music_recommender-A4vLtpgt/lib/python3.6/site-packages/vaex/scopes.py", line 133, in __getitem__
    raise KeyError("Unknown

MemoryError: 

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
lcl = [z.split(" ") for z in cl]

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(cl)]

In [ ]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [ ]:
model.build_vocab(documents)

In [ ]:
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#INFO:MainThread:gensim.models.base_any2vec:training on a 57955880 raw words (115911720 effective words) took 7759.6s, 14938 effective words/s

In [ ]:
x ="Yeah, I'm gonna take my horse to the old town road I'm gonna ride 'til \
I can't no more I'm gonna take my horse to the old town \
road I'm gonna ride 'til I can't no more\
I got the horses in the back\
Horse tack is attached\
Hat is matte black\
Got the boots that's black to match\
Ridin' on a horse, ha\
You can whip your Porsche\
I been in the valley\
You ain't been up off that porch, now"

In [ ]:
x = x.split(" ")

In [ ]:
y = "Hat down, cross town, livin' like a rockstar\
Spent a lot of money on my brand new guitar\
Baby's got a habit: diamond rings and Fendi sports bras\
Ridin' down Rodeo in my Maserati sports car\
Got no stress, I've been through all that\
I'm like a Marlboro Man so I kick on back\
Wish I could roll on back to that old town road\
I wanna ride 'til I can't no more"

In [ ]:
y= y.split(" ")

In [ ]:
x = model.infer_vector(x)

In [ ]:
model.infer_vector(y)

In [ ]:
pj = "Yamborghini chain, rest in peace to my superior\
Hermès link could feed a village in Liberia\
TMZ taking pictures, causin' mad hysteria\
Momma see me on BET and started tearin' up\
I'ma start killin' niggas, how you get that trife?\
I attended Harlem picnics where you risked your life\
Uncle used to skim work, sellin' nicks at night\
I was only 8 years old, watching Nick at Nite\
Uncle Psycho was in that bathroom buggin'!\
Knife to his guts, hope Daddy don't cut him\
Suicidal thoughts brought to me with no advisory\
He was pitchin' dummy, sellin' fiends mad ivory\
Grandma had the arthritis in her hands, bad!\
She was poppin' pills like rappers in society\
I fuck yo bitch for the irony\
I'll send Meechy at yo ho if yo bitch keep eyein' me"

In [ ]:
pj = pj.split(" ")

In [ ]:
pj = model.infer_vector(pj)

In [ ]:
model.docvecs.most_similar([pj],topn=10)

In [ ]:
songs = df.song_identifier.evaluate()

In [ ]:
dff[964697]

In [ ]:
lyrics = dff.lyrics.evaluate()

In [ ]:
lyrics[641956]

In [ ]:
cl[641956]

In [ ]:
documents[641956]

In [ ]:
model.save("lyric_doc2vec")

In [ ]:
model2=Doc2Vec.load("lyric_doc2vec")

In [ ]:
pj1 = "I'ma explain why you probably never seen me\
I'm in a sunken place, no Instagram, I'm watchin' TV\
I think I trade my breakfast, lunch and dinner for some kitty\
Please believe me, I see RiRi, I'ma eat it like panini\
I go dumb up in the broad, hit the walls like graffiti\
Indian burns all up on a nigga wee-wee\
I think I need a foursome, Bella, Kendall, Gigi\
It'd be easy if Reneezy hook it all up on the leezy\
I go crazy in my Yeezy, Kirk Kneezy on the beat\
I told 'em now we finna glow up in the street\
Rappers talk subliminal but they don't talk to me\
Put 'em in a Jersey shore like Pauly D"

In [ ]:
pj1 = pj1.split(" ")
pj1 = model2.infer_vector(pj1)

In [ ]:
model2.docvecs.most_similar([pj1],topn=10)

In [ ]:
z[1532871]

In [ ]:
lyrics = df.lyrics.evaluate()

In [ ]:
lyrics[1532871]

In [ ]:
" ".join(literal_eval(lyrics[1532871]))

# Get edgelist

In [ ]:
df.drop(df.meta_data_1,inplace=True)
df.drop(df.header_links,inplace=True)
df.drop(df.side_table,inplace=True)
df.drop(df.index,inplace=True)

In [ ]:
def edge_list(song_name,associated_data):
    return np.array([(song_name,i) for i in associated_data])

In [ ]:
df.add_virtual_column("nodes",df.meta_data_2.apply(len))

In [ ]:
song = df.song_identifier.evaluate()
data = df.meta_data_2.evaluate()

In [ ]:
df.add_column("edge_list",np.array([edge_list(a,b) for a,b in zip(song,data)]))

In [ ]:
associated_data = df.evaluate(df.meta_data_2)

In [ ]:
edge_list = df.edge_list.evaluate()

In [ ]:
edge_list  = np.array([ i for sublist in edge_list for i in sublist ])

In [ ]:
type(edge_list)

In [ ]:
lyrics

In [ ]:
#!pipenv install networkx --dev

In [ ]:
import networkx as nx

In [ ]:
G = nx.from_edgelist(edge_list)

In [ ]:
g = nx.to_scipy_sparse_matrix(G)

In [ ]:
#!pipenv install scikit-learn --dev

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, n_iter=50, random_state=42)
g_svd = svd.fit_transform(g)  

In [ ]:
G.size()